<img src="../../images/qiskit-heading.gif" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="500 px" align="left">

# Jupyter Tools for Terra

Using Qiskit Terra in a jupyter notebook allows the full power of a web browser to be harnessed to track the status of jobs and view the details of backends using HTML and Javascript.  Here we will demonstrated these tools.  Note that you will need to have `ipywidgets` version `7.3.0` or higher installed for this notebook.  This package is installed by default in Anaaconda Python. 

## Loading the Monitoring Tools

First, let us load the default qiskit routines, and register our IBMQ credentials.

In [ ]:
from qiskit import *
IBMQ.load_accounts(hub=None)

In order to load the Jupyter tools, we need to register them with the Jupyter kernel.  This is done via the `*` import below:

In [ ]:
from qiskit.tools.jupyter import *

The `job_monitor` function will also use Jupyter functionality in asynchronous mode, so lets load it as well:

In [ ]:
from qiskit.tools.monitor import job_monitor

## Tracking Job Status

This is an extension of the `job_monitor` functionality explained in [Monitoring jobs and backends](../terra/backend_monitoring_tools.ipynb).  We encourage you to go through this notebook first, if you have not done so already.

### Monitoring the status of a single job asynchronously

Lets us again build a simple Bell circuit, submit it to a device, and then monitor its status.  However, this time lets monitor the job in async-mode so that we can continue to use our notebook while we wait.

In [ ]:
q = QuantumRegister(2)
c = ClassicalRegister(2)
qc = QuantumCircuit(q, c)

qc.h(q[0])
qc.cx(q[0], q[1])
qc.measure(q, c);

Lets grab the least busy backend

In [ ]:
from qiskit.providers.ibmq import least_busy
backend = least_busy(IBMQ.backends(simulator=False))
backend.name()

Monitor the job using `job_monitor` using async-mode:

In [ ]:
job = execute(qc, backend)
job_monitor(job, monitor_async=True)

It is also possible to monitor the job using the `qiskit_job_status` Jupyter notebook magic.  This method is always asynchronos.

In [ ]:
%%qiskit_job_status
job2 = execute(qc, backend)

Note that, for the `qiskit_job_status` to work, the job returned by `execute` must be stored in a variable so that it may be retrieved by the magic.

### Monitoring many jobs simultaneously

Here we will monitor many jobs sent the the device.  It is best if the jobs are stored in a list to make retrevial easier.

In [ ]:
num_jobs = 5
my_jobs = []
for j in range(num_jobs):
    my_jobs.append(execute(qc, backend))
    job_monitor(my_jobs[j], monitor_async=True)

Or, using magic:

In [ ]:
%%qiskit_job_status
my_jobs2 = []
for j in range(num_jobs):
    my_jobs2.append(execute(qc, backend))

In the magics example, the magic is smart enough to know that the list `my_jobs2` contains jobs, and will automatically extract them and check their status.  We are not limited to using `jobs.append()`, and can use an indexed list or NumPy array as well:

In [ ]:
%%qiskit_job_status
import numpy as np
my_jobs3 = np.empty(num_jobs, dtype=object)
for j in range(num_jobs):
    my_jobs3[j] = execute(qc, backend)

### Changing the interval of status updating

We have already seen in [Monitoring jobs and backends](../terra/backend_monitoring_tools.ipynb) how to change the interval of job status checks.  Here we demo how this works in the magic tools using the `-i` or `--interval` arguments:

In [ ]:
%%qiskit_job_status -i 5
job4 = execute(qc, backend)

In [ ]:
%%qiskit_job_status --interval 5
job5 = execute(qc, backend)

## Rich Display of Backend Details

[Monitoring jobs and backends](../terra/backend_monitoring_tools.ipynb) explored how to view the detailed information on IBM Q devices using text-based functions.  Here we will use the notebook interface to display the device details in a visual widget that are called via magic functions.

To get detailed information on a single backend device, use the `%qiskit_backend_monitor` magic:

In [ ]:
%qiskit_backend_monitor backend

To view all the backends that are available from the `IBMQ` provider (after calling `IBMQ.load_accounts()`), you can use the `backend_overview` magic that runs live in the notebook, and will automatically update itself every minute.

In [ ]:
%qiskit_backend_overview